In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
from snowflake.snowpark.functions import col, count, sum, avg, round
import plotly.express as px

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
session.sql("use database demo").collect()
session.sql("use schema magi_handson").collect()
snowpark_df = session.table("market_info")
snowpark_df.limit(10)

In [ ]:
snowpark_df.describe

In [ ]:
st.subheader("시가총액 상위 5개 ETF")
top_5_snowpark = (snowpark_df
        .with_column("MKCAP_TRILLION", col("MKCAP") / 1e12)
        .order_by(col("MKCAP").desc())
        .limit(5)
        .select("BASE_DT", "ITEM_NM_KOR", "MKCAP_TRILLION", "BASE_IDX_NM")
)
top_5_snowpark

In [ ]:
st.subheader("시가총액 분포")
fig_hist = px.histogram(snowpark_df, x='MKCAP', nbins=10, 
                       title='시가총액 분포',
                       labels={'MKCAP': '시가총액 (원)', 'count': '빈도'})
st.plotly_chart(fig_hist, use_container_width=True)

In [ ]:
st.subheader("매수금액 상위 5개 ETF")
df_with_calculations = (snowpark_df
        .with_column("BUY_AMT_BILLION", col("SUM_BUY_AMT") / 1e8)
        .with_column("SEL_AMT_BILLION", col("SUM_SEL_AMT") / 1e8)
        .with_column("NET_FLOW", col("SUM_BUY_AMT") - col("SUM_SEL_AMT"))
        .with_column("NET_FLOW_BILLION", col("NET_FLOW") / 1e8)
    )
    
# 매수금액 상위 5개 ETF
top_buy_snowpark = (df_with_calculations
    .order_by(col("SUM_BUY_AMT").desc())
    .limit(5)
    .select("BASE_DT", "ITEM_NM_KOR", "BUY_AMT_BILLION")
)

top_buy_snowpark

In [ ]:
st.header("기준지수별 분석")

index_summary = (snowpark_df
    .group_by("BASE_IDX_NM")
    .agg(
        count(col("MKCAP")).alias("MKCAP_count"),
        sum(col("MKCAP")).alias("MKCAP_sum"),
        avg(col("MKCAP")).alias("MKCAP_mean"),
        sum(col("SUM_BUY_AMT")).alias("BUY_sum"),
        avg(col("SUM_BUY_AMT")).alias("BUY_mean"),
        sum(col("SUM_SEL_AMT")).alias("SEL_sum"),
        avg(col("SUM_SEL_AMT")).alias("SEL_mean")
    )
    .select(
        col("BASE_IDX_NM"),
        round(col("MKCAP_count"), 2).alias("MKCAP_count"),
        round(col("MKCAP_sum"), 2).alias("MKCAP_sum"),
        round(col("MKCAP_mean"), 2).alias("MKCAP_mean"),
        round(col("BUY_sum"), 2).alias("BUY_sum"),
        round(col("BUY_mean"), 2).alias("BUY_mean"),
        round(col("SEL_sum"), 2).alias("SEL_sum"),
        round(col("SEL_mean"), 2).alias("SEL_mean")
    )
)

index_summary